In [2]:
from __future__ import division
import numpy as np
from scipy import stats
from scipy import spatial

In [3]:
# Import training data

train = np.genfromtxt('data/trn_data.csv', delimiter=',',skip_header=True)
print train[:5], train.shape

[[  40.582 -119.79    84.929]
 [  41.983 -123.6     87.388]
 [  41.85  -123.97    36.259]
 [  42.07  -123.01    84.913]
 [  41.953 -121.58    66.445]] (414, 3)


In [4]:
stats.describe(train[:,-1])

DescribeResult(nobs=414, minmax=(3.1692999999999998, 134.25), mean=70.239483816425121, variance=717.7203728115777, skewness=-0.14928497285720768, kurtosis=-0.7441244511130987)

In [5]:
test = np.genfromtxt('data/tst_locations.csv', delimiter=',',skip_header=True)
print test[:5], test.shape

[[  41.988 -123.72 ]
 [  41.883 -124.13 ]
 [  41.833 -123.83 ]
 [  41.999 -121.7  ]
 [  41.928 -122.44 ]] (413, 2)


In [6]:
# Matrix of coordinates
X = train[:,:-1]
print X[:5]

[[  40.582 -119.79 ]
 [  41.983 -123.6  ]
 [  41.85  -123.97 ]
 [  42.07  -123.01 ]
 [  41.953 -121.58 ]]


In [7]:
# Matrix of measured values
Y = train[:,-1:]
print Y[:5]

[[ 84.929]
 [ 87.388]
 [ 36.259]
 [ 84.913]
 [ 66.445]]


In [8]:
train[:5,:2]

array([[  40.582, -119.79 ],
       [  41.983, -123.6  ],
       [  41.85 , -123.97 ],
       [  42.07 , -123.01 ],
       [  41.953, -121.58 ]])

In [9]:
test[:2]

array([[  41.988, -123.72 ],
       [  41.883, -124.13 ]])

In [10]:
d = spatial.distance_matrix(train[:5,:2], test[:2])
print d

[[ 4.17393531  4.53080578]
 [ 0.12010412  0.53935146]
 [ 0.2855591   0.16336768]
 [ 0.71471953  1.13550385]
 [ 2.1402862   2.5509606 ]]


In [11]:
l = .4
np.exp(-(d**2) / 2*l**2)

array([[ 0.24814578,  0.1935429 ],
       [ 0.99884667,  0.9769967 ],
       [ 0.99349771,  0.99786716],
       [ 0.95995783,  0.9019921 ],
       [ 0.6931797 ,  0.5941687 ]])

The common covariance formula we will use is the squared exponential:

$$
K_{SE}(x,x') = exp(- \frac{d^2}{2l^2})
$$

where $l$ is the characteristic length-scale of the Gaussian process (we will determine this experimentally).

In [12]:
def covariance(x, y, l):     
    d = spatial.distance_matrix(x,y)
    K = np.exp(-(d**2) / (2*l*l))
    return K

In [13]:
covariance(X[:5], X[:5], 5)

array([[ 1.        ,  0.71922717,  0.68276505,  0.77751614,  0.90332325],
       [ 0.71922717,  1.        ,  0.99691299,  0.99291186,  0.92161658],
       [ 0.68276505,  0.99691299,  1.        ,  0.98078697,  0.8918528 ],
       [ 0.77751614,  0.99291186,  0.98078697,  1.        ,  0.95966426],
       [ 0.90332325,  0.92161658,  0.8918528 ,  0.95966426,  1.        ]])

In [14]:
K = covariance(X,X,5)

In [15]:
K.shape

(414, 414)

In [16]:
covariance(X,test,5).shape

(414, 413)

In [17]:
test.shape

(413, 2)

In [18]:
inv = np.linalg.inv(K+np.var(Y)*np.eye(len(X)))
print inv

[[  1.39519265e-03  -1.02940540e-06  -9.66419602e-07 ...,  -4.42496953e-07
   -3.99192867e-07  -2.47308275e-07]
 [ -1.02940540e-06   1.39504964e-03  -1.62402252e-06 ...,  -4.32657537e-08
   -3.01146406e-08   5.83216262e-09]
 [ -9.66419602e-07  -1.62402252e-06   1.39503827e-03 ...,  -3.36512172e-08
   -2.17938657e-08   9.97798666e-09]
 ..., 
 [ -4.42496953e-07  -4.32657537e-08  -3.36512172e-08 ...,   1.39503856e-03
   -1.64051863e-06  -1.59811245e-06]
 [ -3.99192867e-07  -3.01146406e-08  -2.17938657e-08 ...,  -1.64051863e-06
    1.39502215e-03  -1.63320096e-06]
 [ -2.47308275e-07   5.83216262e-09   9.97798666e-09 ...,  -1.59811245e-06
   -1.63320096e-06   1.39495739e-03]]


In [19]:
K_test = covariance(test, X, 5)
print K_test.dot(inv)

[[  7.21495340e-04   1.16486851e-03   1.16707158e-03 ...,   2.74683965e-05
    1.84810064e-05  -5.84265253e-06]
 [  6.70271748e-04   1.16313215e-03   1.17437465e-03 ...,   1.92054143e-05
    1.13247480e-05  -9.43343749e-06]
 [  7.10594127e-04   1.16174982e-03   1.16737983e-03 ...,   2.83565030e-05
    1.93448031e-05  -5.11932335e-06]
 ..., 
 [  1.30799477e-04  -1.35778651e-05  -1.57913813e-05 ...,   1.09169375e-03
    1.12436192e-03   1.21409665e-03]
 [  3.01770211e-04   2.81417212e-05   2.18622452e-05 ...,   1.17009327e-03
    1.17585407e-03   1.15451317e-03]
 [  2.67378647e-04   1.48695103e-05   9.31462808e-06 ...,   1.17641493e-03
    1.18712026e-03   1.18359453e-03]]


In [20]:
print K_test.dot(inv).dot(Y)

[[ 15.66505457]
 [ 15.10838074]
 [ 15.75805578]
 [ 18.18120879]
 [ 17.66140331]
 [ 16.53887742]
 [ 17.97902954]
 [ 17.24764709]
 [ 18.59263879]
 [ 17.69746621]
 [ 18.63284448]
 [ 18.83833808]
 [ 18.95536732]
 [ 18.98196028]
 [ 18.97792902]
 [ 17.39124283]
 [ 19.512018  ]
 [ 18.76422715]
 [ 19.47108461]
 [ 19.46519434]
 [ 19.48233372]
 [ 19.43316659]
 [ 18.85568789]
 [ 17.30812575]
 [ 18.53044084]
 [ 19.01381719]
 [ 18.6022629 ]
 [ 18.31986725]
 [ 18.27937448]
 [ 20.13138082]
 [ 20.66773312]
 [ 19.80905782]
 [ 20.73924918]
 [ 17.17434571]
 [ 20.43461088]
 [ 20.0524674 ]
 [ 20.42166852]
 [ 20.4791339 ]
 [ 21.33481327]
 [ 21.02073312]
 [ 21.63868018]
 [ 21.15782677]
 [ 21.16151852]
 [ 21.80215366]
 [ 21.78375995]
 [ 21.96544149]
 [ 22.41480939]
 [ 22.80008911]
 [ 22.55016246]
 [ 22.97236855]
 [ 22.6354471 ]
 [ 22.54878738]
 [ 22.46603575]
 [ 22.91777129]
 [ 22.46191132]
 [ 22.68757978]
 [ 22.56621202]
 [ 22.86117257]
 [ 23.26652021]
 [ 22.94006886]
 [ 22.8410028 ]
 [ 22.68353334]
 [ 22.34

In [21]:
def predictive_mean(x, x_test,y,l,indices=False):
    
    K_xtest_x = covariance(x_test, x, l)

    K = covariance(x, x, l)
    
    sigma_sq_I = np.var(y)*np.eye(len(x))
    inv = np.linalg.inv(K+sigma_sq_I)
    
    predictions = K_xtest_x.dot(inv).dot(y)

    if indices:
        return np.concatenate([x_test, predictions], axis=1)
    else:
        return predictions

In [22]:
predictions = predictive_mean(X, test, Y, 5)

In [23]:
predictions[:5]

array([[ 15.66505457],
       [ 15.10838074],
       [ 15.75805578],
       [ 18.18120879],
       [ 17.66140331]])

In [24]:
test[:5]

array([[  41.988, -123.72 ],
       [  41.883, -124.13 ],
       [  41.833, -123.83 ],
       [  41.999, -121.7  ],
       [  41.928, -122.44 ]])

In [25]:
np.concatenate([test, predictions], axis=1)

array([[  41.988     , -123.72      ,   15.66505457],
       [  41.883     , -124.13      ,   15.10838074],
       [  41.833     , -123.83      ,   15.75805578],
       ..., 
       [  32.74      , -114.88      ,   10.57755354],
       [  34.208     , -116.62      ,   16.17741648],
       [  34.017     , -116.18      ,   15.0423128 ]])

In [26]:
np.genfromtxt('data/grid.csv', delimiter=',')

array([[  38.5  , -120.8  ],
       [  38.516, -120.8  ],
       [  38.533, -120.8  ],
       ..., 
       [  39.267, -119.8  ],
       [  39.284, -119.8  ],
       [  39.3  , -119.8  ]])

In [27]:
def make_grid(bounding_box, ncell):
    xmax, xmin, ymax, ymin = bounding_box
    xgrid = np.linspace(xmin, xmax, ncell)
    ygrid = np.linspace(ymin, ymax, ncell)
    mX, mY = np.meshgrid(xgrid, ygrid)
    ngridX = mX.reshape(ncell*ncell, 1)
    ngridY = mY.reshape(ncell*ncell, 1)
    return np.concatenate((ngridX, ngridY), axis=1)

In [28]:
bounding_box = [38.3, 39.3, -120.0, -121.0]

In [29]:
xmax, xmin, ymax, ymin = bounding_box

In [30]:
ncell=3
xgrid = np.linspace(xmin, xmax, ncell)
ygrid = np.linspace(ymin, ymax, ncell)

In [31]:
mX, mY = np.meshgrid(xgrid, ygrid)

In [32]:
ngridY = mY.reshape(ncell*ncell, 1)

In [33]:
ngridY

array([[-121. ],
       [-121. ],
       [-121. ],
       [-120.5],
       [-120.5],
       [-120.5],
       [-120. ],
       [-120. ],
       [-120. ]])

In [34]:
grid = make_grid(bounding_box=bounding_box, ncell=5)

In [35]:
predictive_mean(x=X, x_test=grid, y=Y, l=1.3, indices=True)

array([[  39.3       , -121.        ,   11.4736564 ],
       [  39.05      , -121.        ,   11.70631744],
       [  38.8       , -121.        ,   11.78213539],
       [  38.55      , -121.        ,   11.69839965],
       [  38.3       , -121.        ,   11.45895916],
       [  39.3       , -120.75      ,   11.56751267],
       [  39.05      , -120.75      ,   11.8981569 ],
       [  38.8       , -120.75      ,   12.06558441],
       [  38.55      , -120.75      ,   12.0625873 ],
       [  38.3       , -120.75      ,   11.8898199 ],
       [  39.3       , -120.5       ,   11.44561187],
       [  39.05      , -120.5       ,   11.86923302],
       [  38.8       , -120.5       ,   12.12909989],
       [  38.55      , -120.5       ,   12.21350918],
       [  38.3       , -120.5       ,   12.11949027],
       [  39.3       , -120.25      ,   11.0989767 ],
       [  39.05      , -120.25      ,   11.60524441],
       [  38.8       , -120.25      ,   11.95354197],
       [  38.55      , -120.

In [41]:
results = []
for i in np.arange(1,100):
    i = i/10
    r = predictive_mean(x=X, x_test=grid, y=Y, l=i, indices=True)
    results.append(r[5,-1])    

In [42]:
results

[0.39125168292487678,
 0.90497053928609139,
 1.6947945064795906,
 2.6856959237248885,
 3.7495383978088115,
 4.8020023541819885,
 5.8266254515193623,
 6.8326917004939656,
 7.8269911912441019,
 8.8069568637335554,
 9.7637299995439104,
 10.686866177237404,
 11.567512671115932,
 12.399736988465031,
 13.180616225359969,
 13.909726753915978,
 14.588455140203981,
 15.219347412186734,
 15.805583138649201,
 16.350591033880992,
 16.857791006727595,
 17.330436524902417,
 17.771530401349832,
 18.183790775913408,
 18.56964908144829,
 18.931266657536664,
 19.270560792740945,
 19.589234180169356,
 19.888804119664517,
 20.170629436598972,
 20.435934172765954,
 20.685827785803873,
 20.92132199078954,
 21.143344584329018,
 21.352750676200376,
 21.550331764668805,
 21.736823061736199,
 21.912909425075679,
 22.079230197407707,
 22.236383199237736,
 22.384928071201927,
 22.525389119475872,
 22.658257782135745,
 22.783994805610742,
 22.903032197645349,
 23.015775005584718,
 23.12260295539604,
 23.2238719768

In [37]:
print train[(train[:,0] < 39.3) & (train[:,0] > 38.3) & (train[:,1] > -121.0) & (train[:,1] < -120.0)]

[[  39.217 -120.98    72.675]
 [  39.264 -120.77    70.799]
 [  39.133 -120.95    39.807]
 [  39.283 -120.7     99.279]
 [  39.276 -120.71    77.494]
 [  39.144 -120.51   134.25 ]
 [  39.094 -120.58    79.799]
 [  39.075 -120.56    93.02 ]
 [  38.983 -120.32    85.883]
 [  38.925 -120.79    62.365]
 [  38.903 -120.38    92.958]
 [  38.805 -120.21   105.83 ]
 [  38.784 -120.31    90.731]
 [  38.747 -120.07   117.34 ]
 [  38.71  -120.04   124.12 ]
 [  38.695 -120.82    80.925]
 [  38.678 -120.12    95.967]
 [  38.718 -120.56    77.387]
 [  38.45  -120.48    75.792]
 [  38.39  -120.65    86.072]
 [  38.375 -120.19   100.51 ]
 [  39.182 -120.12   103.61 ]
 [  39.172 -120.15   102.65 ]
 [  39.001 -120.14    94.788]
 [  38.849 -120.08   105.81 ]]
